In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
PREFIX_INPUT = "/kaggle/input/playground-series-s4e7"
PREFIX_OUTPUT = "/kaggle/working"

In [3]:
train = pd.read_csv(f'{PREFIX_INPUT}/train.csv')
test = pd.read_csv(f'{PREFIX_INPUT}/test.csv')

In [4]:
# train.columns

In [5]:
# train.head()

## Data preprocessing

In [6]:
# train["Response"].value_counts()

In [7]:
# train.isna().sum()

Note: 
+ no null
+ label: Response 
+ id: no use
+ Gender: binary 
+ Age: -> change to category
+ Driving_License: binary  
+ Region_Code: change to category
+ previous_insured: binary 
+ Vehicle_Damage: binary
+ Vehicle_Damage: category
+ Annual_Premium: numerical 
+ Policy_Sales_Channel: -> change to category
+ vintage: -> change to category

Important: 
+ this is imbalance dataset

## EDA & Feature Selection

### Binary cols

In [8]:
import seaborn as sns 
import matplotlib.pyplot as plt

In [9]:
# train.groupby('Previously_Insured')["Response"].value_counts()

In [10]:
# train.groupby('Gender')["Response"].value_counts()

In [11]:
# train.groupby('Driving_License')["Response"].value_counts()

In [12]:
# train.groupby('Vehicle_Age')["Response"].value_counts()

In [13]:
# train.groupby('Vehicle_Damage')["Response"].value_counts()

Note: 
+ people with no insure has higher chance to respond
+ Male has slightly higher rate of response
+ people without license is not likely to respond
+ the more people ride, the more they respond
+ people with vihicle damage arre likely to respond

### Numerical cols

Age, Region_Code, Annual_Premium, Policy_Sales_Channel, Vintage


In [14]:
def plot_all_cols(data,cols):
    id_res = (data["Response"] == 1)
    id_no_res = (data["Response"] == 0)
    l = len(cols)
    plt.figure(figsize=(20,5*l))
    for i,col in enumerate(cols):
        print(col)
        plt.subplot(l,2,i*2+1)
        value,count = np.unique(data[col].loc[id_res],return_counts=True)
        plt.bar(value,count)
        plt.subplot(l,2,i*2+2)
        value,count = np.unique(data[col].loc[id_no_res],return_counts=True)
        plt.bar(value,count,color='r')
    plt.tight_layout()
        

In [15]:
# plot_all_cols(train,["Age","Region_Code","Policy_Sales_Channel"])

In [16]:
def cal_pct_diff(data,col):
    id_res = (data["Response"] == 1)
    id_no_res = (data["Response"] == 0)
    value_res,count_res = np.unique(data[col].loc[id_res],return_counts=True)
    value_no_res,count_no_res = np.unique(data[col].loc[id_no_res],return_counts=True)
    res = {}
    for i in range(len(value_res)):
        res[value_res[i]] = count_res[i]/count_no_res[i]
    return res

In [17]:
pct_res = (train["Response"] == 1).sum()/(train["Response"] == 0).sum()
pct_res

0.14024733444542023

In [18]:
dict_pct = cal_pct_diff(train,"Vintage")
keys = np.array(list(dict_pct.keys()))
values = np.array(list(dict_pct.values()))
# plt.figure(figsize=(20,5))
# plt.bar(keys,values)
# plt.plot([10,300],[pct_res,pct_res],'r')
# plt.plot([10,300],[pct_res,pct_res],'r')

In [19]:
def draw_plot_range_pct(range_pct,keys,values,pct_res=pct_res):
    plt.bar(keys,sorted(values))
    plt.plot([min(keys),max(keys)],[pct_res,pct_res],'r')
    for pct in range_pct:
        plt.plot([min(keys),max(keys)],[pct,pct],'g')

In [20]:
range_pct = [0.08,0.1,0.12,0.17,0.22,0.28,0.3,0.33,0.37]
# draw_plot_range_pct(range_pct,keys,values)

In [21]:
def create_groups(df, column_name,keys,values, range_pct):    
    bins = [float('-inf')] + range_pct + [float('inf')]
    cutten_groups = []
    
    for i in range(1, len(bins)):
        idx = (values > bins[i-1]) & (values <= bins[i])
        cutten_groups.append(keys[idx])
    
    # Check if any key is missing
    total_grouped = sum(len(g) for g in cutten_groups)
    if total_grouped != len(keys):
        print(f"Warning: {len(keys) - total_grouped} keys are missing from groups")
    
    dict_cutten_groups = {}
    for idx, group in enumerate(cutten_groups):
        for value in group:
            dict_cutten_groups[value] = f'Group{idx}'
    
    new_column_name = f'{column_name}Group'
    df[new_column_name] = df[column_name].map(dict_cutten_groups).fillna('Unknown')
    
    return df

In [22]:
create_groups(train,"Vintage",keys,values,range_pct)
create_groups(test,"Vintage",keys,values,range_pct)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,VintageGroup
0,11504798,Female,20,1,47.0,0,< 1 Year,No,2630.0,160.0,228,Group0
1,11504799,Male,47,1,28.0,0,1-2 Year,Yes,37483.0,124.0,123,Group5
2,11504800,Male,47,1,43.0,0,1-2 Year,Yes,2630.0,26.0,271,Group3
3,11504801,Female,22,1,47.0,1,< 1 Year,No,24502.0,152.0,115,Group3
4,11504802,Male,51,1,19.0,0,1-2 Year,No,34115.0,124.0,148,Group8
...,...,...,...,...,...,...,...,...,...,...,...,...
7669861,19174659,Male,57,1,28.0,0,1-2 Year,Yes,51661.0,124.0,109,Group3
7669862,19174660,Male,28,1,50.0,1,< 1 Year,No,25651.0,152.0,184,Group2
7669863,19174661,Male,47,1,33.0,1,1-2 Year,No,2630.0,138.0,63,Group2
7669864,19174662,Male,30,1,28.0,0,< 1 Year,Yes,38866.0,124.0,119,Group2


In [23]:
# train["VintageGroup"].value_counts()

In [24]:
dict_pct = cal_pct_diff(train,"Region_Code")
keys = np.array(list(dict_pct.keys()))
values = np.array(list(dict_pct.values()))
# idx = values < 0.9
# plt.bar(keys[idx],values[idx])
# plt.bar(keys[~idx],1,color='r')
# plt.plot([0,52],[pct_res,pct_res],'r')

In [25]:
# values_tmp = values.copy() 
# values_tmp[~idx] = 0.9

In [26]:
range_pct = [0.04,0.07,0.11,0.2,0.26,0.42,0.6,1]

In [27]:
# draw_plot_range_pct(range_pct=range_pct,keys=keys,values=values_tmp)

In [28]:
range_pct += [3] # add for the region code 39 and 40

In [29]:
create_groups(train,"Region_Code",keys,values,range_pct)
create_groups(test,"Region_Code",keys,values,range_pct)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,VintageGroup,Region_CodeGroup
0,11504798,Female,20,1,47.0,0,< 1 Year,No,2630.0,160.0,228,Group0,Group0
1,11504799,Male,47,1,28.0,0,1-2 Year,Yes,37483.0,124.0,123,Group5,Group4
2,11504800,Male,47,1,43.0,0,1-2 Year,Yes,2630.0,26.0,271,Group3,Group6
3,11504801,Female,22,1,47.0,1,< 1 Year,No,24502.0,152.0,115,Group3,Group0
4,11504802,Male,51,1,19.0,0,1-2 Year,No,34115.0,124.0,148,Group8,Group4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7669861,19174659,Male,57,1,28.0,0,1-2 Year,Yes,51661.0,124.0,109,Group3,Group4
7669862,19174660,Male,28,1,50.0,1,< 1 Year,No,25651.0,152.0,184,Group2,Group5
7669863,19174661,Male,47,1,33.0,1,1-2 Year,No,2630.0,138.0,63,Group2,Group2
7669864,19174662,Male,30,1,28.0,0,< 1 Year,Yes,38866.0,124.0,119,Group2,Group4


In [30]:
# train["Policy_Sales_Channel"].value_counts()

In [31]:
dict_pct = cal_pct_diff(train,"Policy_Sales_Channel")
keys = np.array(list(dict_pct.keys()))
values = np.log1p(np.array(list(dict_pct.values())))
# idx = values < 0.4
# # plt.bar(keys,values)
# plt.bar(keys[idx],values[idx])
# # plt.bar(keys[~idx],1,color='r')
# plt.plot([min(keys),max(keys)],[pct_res,pct_res],'r')

In [32]:
# values_tmp = values.copy() 
# values_tmp[~idx] = 0.4

In [33]:
range_pct = [0.05,0.1,0.2,0.25,0.39]

In [34]:
# draw_plot_range_pct(range_pct=range_pct,keys=keys,values=values_tmp)

In [35]:
# idx2 = values > 0.4
# # plt.bar(keys,values)
# plt.bar(keys[idx2],values[idx2])
# # plt.bar(keys[~idx],1,color='r')
# plt.plot([min(keys),max(keys)],[pct_res,pct_res],'r')

In [36]:
range_pct_2 = [1.1,2,2.7,3.3,4,4.9,6.8]
# draw_plot_range_pct(range_pct=range_pct_2,keys=keys,values=values)

In [37]:
final_range_pct = range_pct + range_pct_2

In [38]:
create_groups(train,"Policy_Sales_Channel",keys,values,final_range_pct)
create_groups(test,"Policy_Sales_Channel",keys,values,range_pct)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,VintageGroup,Region_CodeGroup,Policy_Sales_ChannelGroup
0,11504798,Female,20,1,47.0,0,< 1 Year,No,2630.0,160.0,228,Group0,Group0,Group5
1,11504799,Male,47,1,28.0,0,1-2 Year,Yes,37483.0,124.0,123,Group5,Group4,Group5
2,11504800,Male,47,1,43.0,0,1-2 Year,Yes,2630.0,26.0,271,Group3,Group6,Group5
3,11504801,Female,22,1,47.0,1,< 1 Year,No,24502.0,152.0,115,Group3,Group0,Group5
4,11504802,Male,51,1,19.0,0,1-2 Year,No,34115.0,124.0,148,Group8,Group4,Group5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7669861,19174659,Male,57,1,28.0,0,1-2 Year,Yes,51661.0,124.0,109,Group3,Group4,Group5
7669862,19174660,Male,28,1,50.0,1,< 1 Year,No,25651.0,152.0,184,Group2,Group5,Group5
7669863,19174661,Male,47,1,33.0,1,1-2 Year,No,2630.0,138.0,63,Group2,Group2,Group0
7669864,19174662,Male,30,1,28.0,0,< 1 Year,Yes,38866.0,124.0,119,Group2,Group4,Group5


In [39]:
# train["Policy_Sales_ChannelGroup"].value_counts()

In [40]:
dict_pct = cal_pct_diff(train,"Age")
keys = np.array(list(dict_pct.keys()))
values = np.array(list(dict_pct.values()))
# plt.figure(figsize=(20,5))
# plt.bar(keys,values)
# plt.plot([20,84],[pct_res,pct_res],'r')

In [41]:
range_pct = [0.04,0.07,0.12,0.16,0.205,0.235,0.265,0.287]

In [42]:
# draw_plot_range_pct(range_pct=range_pct,keys=keys,values=values)

In [43]:
create_groups(train,"Age",keys,values,range_pct)
create_groups(test,"Age",keys,values,range_pct)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,VintageGroup,Region_CodeGroup,Policy_Sales_ChannelGroup,AgeGroup
0,11504798,Female,20,1,47.0,0,< 1 Year,No,2630.0,160.0,228,Group0,Group0,Group5,Group0
1,11504799,Male,47,1,28.0,0,1-2 Year,Yes,37483.0,124.0,123,Group5,Group4,Group5,Group7
2,11504800,Male,47,1,43.0,0,1-2 Year,Yes,2630.0,26.0,271,Group3,Group6,Group5,Group7
3,11504801,Female,22,1,47.0,1,< 1 Year,No,24502.0,152.0,115,Group3,Group0,Group5,Group0
4,11504802,Male,51,1,19.0,0,1-2 Year,No,34115.0,124.0,148,Group8,Group4,Group5,Group5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7669861,19174659,Male,57,1,28.0,0,1-2 Year,Yes,51661.0,124.0,109,Group3,Group4,Group5,Group4
7669862,19174660,Male,28,1,50.0,1,< 1 Year,No,25651.0,152.0,184,Group2,Group5,Group5,Group2
7669863,19174661,Male,47,1,33.0,1,1-2 Year,No,2630.0,138.0,63,Group2,Group2,Group0,Group7
7669864,19174662,Male,30,1,28.0,0,< 1 Year,Yes,38866.0,124.0,119,Group2,Group4,Group5,Group3


In [44]:
# train["AgeGroup"].value_counts()

In [45]:
train["Annual_log"] = train["Annual_Premium"].apply(np.log1p)
test["Annual_log"] = test["Annual_Premium"].apply(np.log1p)

In [46]:
# train["Annual_log"].describe()

In [47]:
bins = [float('-inf'), 8.0, 8.5,9.0,9.5,10,10.5,11,11.5,12,12.5,13,float('inf')]
labels = ['-8','8-8.5','8.5-9','9-9.5','9.5-10','10-10.5','10.5-11','11-11.5','11.5-12','12-12.5','12.5-13','13+']
train.loc[:, "AnnualGroup"]= pd.cut(train["Annual_log"],bins=bins,labels=labels,right=True, include_lowest=True)
test.loc[:, "AnnualGroup"]= pd.cut(test["Annual_log"],bins=bins,labels=labels,right=True, include_lowest=True)

In [48]:
# grouped_age= train.groupby(['AnnualGroup', 'Response']).size().unstack(fill_value=0)
# grouped_age.plot(kind='bar', stacked=False, color=['red', 'green'])

In [49]:
# del grouped_age

In [50]:
dict_pct = cal_pct_diff(train,"AnnualGroup")
keys = np.array(list(dict_pct.keys()))
values = np.array(list(dict_pct.values()))
# plt.figure(figsize=(20,5))
# plt.bar(keys,values)
# plt.plot([0,len(keys)-1],[pct_res,pct_res],'r')

In [51]:
del dict_pct
del keys
del values

In [52]:
# train["AnnualGroup"].value_counts()

In [53]:
# train

## Feature engineering

In [54]:
columns = ["Gender","Driving_License","Previously_Insured","Vehicle_Age","Vehicle_Damage","Region_CodeGroup","Policy_Sales_ChannelGroup","VintageGroup","AgeGroup","AnnualGroup","Annual_Premium"]
label = ["Response"]

In [55]:
df_train = train[columns+label]
X_train = df_train.drop(columns=label)
y_train = df_train[label]
X_test = test[columns]

In [56]:
del train

In [57]:
binary_cols = ["Gender","Driving_License","Previously_Insured","Vehicle_Damage"]
multi_cols = ["Vehicle_Age","Region_CodeGroup","Policy_Sales_ChannelGroup","VintageGroup","AgeGroup","AnnualGroup"]
num_cols = ["Annual_Premium"]

In [58]:
# One hot encoding
X_train = pd.get_dummies(X_train, columns=multi_cols)
X_train = pd.get_dummies(X_train, columns=binary_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=multi_cols)
X_test = pd.get_dummies(X_test, columns=binary_cols, drop_first=True)


In [59]:
X_train.columns

Index(['Annual_Premium', 'Vehicle_Age_1-2 Year', 'Vehicle_Age_< 1 Year',
       'Vehicle_Age_> 2 Years', 'Region_CodeGroup_Group0',
       'Region_CodeGroup_Group1', 'Region_CodeGroup_Group2',
       'Region_CodeGroup_Group3', 'Region_CodeGroup_Group4',
       'Region_CodeGroup_Group5', 'Region_CodeGroup_Group6',
       'Region_CodeGroup_Group7', 'Region_CodeGroup_Group8',
       'Region_CodeGroup_Group9', 'Region_CodeGroup_Unknown',
       'Policy_Sales_ChannelGroup_Group0', 'Policy_Sales_ChannelGroup_Group1',
       'Policy_Sales_ChannelGroup_Group10',
       'Policy_Sales_ChannelGroup_Group11',
       'Policy_Sales_ChannelGroup_Group12', 'Policy_Sales_ChannelGroup_Group2',
       'Policy_Sales_ChannelGroup_Group3', 'Policy_Sales_ChannelGroup_Group4',
       'Policy_Sales_ChannelGroup_Group5', 'Policy_Sales_ChannelGroup_Group6',
       'Policy_Sales_ChannelGroup_Group7', 'Policy_Sales_ChannelGroup_Group8',
       'Policy_Sales_ChannelGroup_Group9', 'Policy_Sales_ChannelGroup_Unknown'

In [60]:
# Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.fit_transform(X_test[num_cols])

In [61]:
# concat_df_train = pd.concat([X_train,y_train],axis=1) 

In [62]:
# corr_matrix = concat_df_train.corr()

In [63]:
# corr_threshold  = 0.1
# high_corr_cols = corr_matrix[abs(corr_matrix['Response']) > corr_threshold].index
# sorted_corr_matrix = concat_df_train[high_corr_cols].corr()
# sorted_corr_matrix['Response'].sort_values(ascending=False)
# sns.heatmap(sorted_corr_matrix, annot=True,cmap='coolwarm')


In [64]:
# del corr_matrix
# del sorted_corr_matrix

## Choosing Model

### Split val-train

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42,stratify=y_train)

In [66]:
y_train.value_counts(), y_val.value_counts()

(Response
 0           9080765
 1           1273553
 Name: count, dtype: int64,
 Response
 0           1008974
 1            141506
 Name: count, dtype: int64)

### RandomForest

Because the dataset is too large, so I will use Chunking technique.

In [67]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import roc_auc_score, f1_score

# rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, warm_start=True)
# chunk_size = 100000  # Adjust based on your available memory
# for i in range(0, len(X_train), chunk_size):
#     X_chunk = X_train[i:i+chunk_size]
#     y_chunk = y_train[i:i+chunk_size].values.reshape(-1)
#     rf_classifier.n_estimators += 10  # Grow 10 new trees each iteration
#     rf_classifier.fit(X_chunk, y_chunk)

# y_pred = rf_classifier.predict(X_val)

# # Evaluate the model
# auc = roc_auc_score(y_val, y_pred)
# f1 = f1_score(y_val, y_pred)
# print(f'AUC: {auc:.4f}')
# print(f'F1 Score: {f1:.4f}')

In [68]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import roc_auc_score, f1_score

# rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, warm_start=True)
# rf_classifier.fit(X_train, y_train.values.reshape(-1))
# y_pred = rf_classifier.predict(X_val)

# # Evaluate the model
# auc = roc_auc_score(y_val, y_pred)
# f1 = f1_score(y_val, y_pred)
# print(f'AUC: {auc:.4f}')
# print(f'F1 Score: {f1:.4f}')

In [69]:
# import joblib
# joblib.dump(rf_classifier, f'{PREFIX_OUTPUT}/rf_classifier_model.joblib')

### Logistic Regression

Using partial_fit of SGD

In [70]:
# from sklearn.linear_model import SGDClassifier
# from sklearn.metrics import roc_auc_score, f1_score
# clf = SGDClassifier(loss='log_loss', random_state=42)

# batch_size = 1000000
# for i in range(0, X_train.shape[0], batch_size):
#     print(f'Training batch {i // batch_size + 1}/{X_train.shape[0] // batch_size}')
#     X_batch = X_train[i:i + batch_size]
#     y_batch = y_train[i:i + batch_size].values.reshape(-1)
#     clf.partial_fit(X_batch, y_batch, classes=np.unique(y_train))

# # Predict on the validation set
# y_pred = clf.predict(X_val)

# # Evaluate the model
# auc = roc_auc_score(y_val, y_pred)
# f1 = f1_score(y_val, y_pred)
# print(f'AUC: {auc:.4f}')
# print(f'F1 Score: {f1:.4f}')

In [71]:
import xgboost as xgb

In [72]:
# change name of Vehicle_Age_<1 year and Vehicle_Age_> 2 Years
X_train = X_train.rename(columns={
    'Vehicle_Age_< 1 Year': 'Vehicle_Age_LessThanOneYear',
    'Vehicle_Age_> 2 Years': 'Vehicle_Age_MoreThanTwoYear',
    'Vehicle_Age_1-2 Year': 'Vehicle_Age_1-TwoYear'
})
X_val = X_val.rename(columns={
    'Vehicle_Age_< 1 Year': 'Vehicle_Age_LessThanOneYear',
    'Vehicle_Age_> 2 Years': 'Vehicle_Age_MoreThanTwoYear',
    'Vehicle_Age_1-2 Year': 'Vehicle_Age_1-TwoYear'
})
X_test = X_test.rename(columns={
    'Vehicle_Age_< 1 Year': 'Vehicle_Age_LessThanOneYear',
    'Vehicle_Age_> 2 Years': 'Vehicle_Age_MoreThanTwoYear',
    'Vehicle_Age_1-2 Year': 'Vehicle_Age_1-TwoYear'
})

In [73]:
test_cols = X_test.columns
train_cols = X_train.columns
set1 = set(test_cols.tolist())
set2 = set(train_cols.tolist())
missing_cols = list(set2.difference(set1))

In [74]:
len(test_cols),len(train_cols)

(57, 65)

In [75]:
missing_cols

['Policy_Sales_ChannelGroup_Group11',
 'Policy_Sales_ChannelGroup_Group10',
 'Policy_Sales_ChannelGroup_Group6',
 'Policy_Sales_ChannelGroup_Group9',
 'Policy_Sales_ChannelGroup_Group12',
 'Policy_Sales_ChannelGroup_Group7',
 'Region_CodeGroup_Unknown',
 'Policy_Sales_ChannelGroup_Group8']

In [76]:
for col in missing_cols: 
    X_test[col] = False 

In [77]:
len(X_test.columns)

65

In [78]:
X_test = X_test[sorted(X_test.columns)]
X_train = X_train[sorted(X_train.columns)]
X_val = X_val[sorted(X_val.columns)]

In [79]:
dtest = xgb.DMatrix(X_test)

In [80]:
from sklearn.model_selection import StratifiedKFold


In [81]:
from sklearn.metrics import roc_auc_score, f1_score

In [83]:
import warnings
warnings.simplefilter('ignore')


In [84]:
aucs = []
preds = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, valid_idx) in enumerate(skf.split(X_train, y_train)):
    print(f'### Fold {fold+1} Training ###')
    X_train_fold = X_train.iloc[train_idx]
    y_train_fold = y_train.iloc[train_idx]
    X_val_fold = X_train.iloc[valid_idx]
    y_val_fold = y_train.iloc[valid_idx]
    
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'random_state': 42,
        'tree_method': 'hist',  # Use GPU acceleration
        'device': "cuda"  # Use the first GPU. Change if you want to use a different GPU
    }
    print("Start changing type to DMatrix...")
    dtrain = xgb.DMatrix(X_train_fold, label=y_train_fold)
    del X_train_fold
    del y_train_fold
    dval = xgb.DMatrix(X_val_fold)
    del X_val_fold
    
    print("Start training...")
    bst = xgb.train(params, dtrain, num_boost_round=100, evals=[(dval, 'eval')], early_stopping_rounds=50, verbose_eval=100)
    
    print("Start predicting val...")
    # Evaluate the model
    y_pred_proba = bst.predict(dval)
    auc_score = roc_auc_score(y_val_fold, y_pred_proba)
    del y_pred_proba
    del y_val_fold
    aucs.append(auc_score)
    print(f'Fold {fold+1} AUC: {auc_score:.5f}\n')
    
    # Pred test set
    print("Start predicting test...")
    dtest = xgb.DMatrix(X_test)  # Assuming you have X_test defined
    y_test_pred_proba = bst.predict(dtest)
    submission = test[['id']]
    submission['Response'] = y_test_pred_proba
    submission.to_csv(f'submission_{fold}.csv', index=False)
    del submission
    del y_test_pred_proba
    del bst
    
    

print(f'\nOverall AUC: {np.mean(aucs):.5f} +/- {np.std(aucs):.5f}')

### Fold 1 Training ###
Start changing type to DMatrix...
Start training...
[0]	eval-auc:nan
[50]	eval-auc:nan
Start predicting val...
Fold 1 AUC: 0.87100

Start predicting test...
### Fold 2 Training ###
Start changing type to DMatrix...
Start training...
[0]	eval-auc:nan
[49]	eval-auc:nan
Start predicting val...
Fold 2 AUC: 0.87115

Start predicting test...
### Fold 3 Training ###
Start changing type to DMatrix...
Start training...
[0]	eval-auc:nan
[49]	eval-auc:nan
Start predicting val...
Fold 3 AUC: 0.87114

Start predicting test...
### Fold 4 Training ###
Start changing type to DMatrix...
Start training...
[0]	eval-auc:nan
[50]	eval-auc:nan
Start predicting val...
Fold 4 AUC: 0.87132

Start predicting test...
### Fold 5 Training ###
Start changing type to DMatrix...
Start training...
[0]	eval-auc:nan
[49]	eval-auc:nan
Start predicting val...
Fold 5 AUC: 0.87156

Start predicting test...

Overall AUC: 0.87123 +/- 0.00019


In [90]:
submissions = []
for i in range(0,5):
    submission = pd.read_csv(f"{PREFIX_OUTPUT}/submission_{i}.csv")
    submissions.append(submission["Response"])

In [94]:
final_submission = test[['id']]
final_submission['Response'] = np.mean(submissions, axis=0)

In [96]:
final_submission.to_csv(f'{PREFIX_OUTPUT}/submission.csv', index=False)

In [99]:
final_submission

,id,Response
0,11504798,0.008123
1,11504799,0.395032
2,11504800,0.221034
3,11504801,0.000253
4,11504802,0.065001
...,...,...
7669861,19174659,0.226091
7669862,19174660,0.000328
7669863,19174661,0.000533
7669864,19174662,0.382057
